# Рекомендательная ALS система

## Для пользователей, у которых уже есть как минимум 5 взаимодействий

In [8]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 50)  # Показывать до 50 колонок
pd.set_option('display.max_colwidth', 100)  # Показывать до 50 колонок

In [9]:
data = pd.read_excel("../cuprum_3.xlsx", sheet_name="Лист4") #cuprum/Лист4/Лист2/Лист1
data

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,esb_ehr_id,patientnet_ehr_id,пол,возраст,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,175881,ed4cb6d0-fc26-43f7-8322-292adc7e6f09,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,38203,703c46d2-b15f-4d00-b4da-7698eca126a3,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,82267,6e97b34a-3a29-48e7-86b1-d63557cf298a,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,33336,bebd6f34-46ef-4f1c-8dbd-628bbfdae108,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,336371,6cdfeb1f-8a4f-457c-a236-fc1d478ae716,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,37822,09789d90-2f43-4c78-b29c-661ca30ca5c5,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30836,9608800,CLICKED,606dd15f70bc473f234d0092,1326732,2024-07-30,1326732,392685,2acb4b14-f65b-4170-9d24-d042a53280b2,2,21,2004-02-12,Полиамория и сологамия: какими бывают современные отношения,https://cuprum.media/lifestyle/romantic-relationship?from=avaapp,10308.0,2021-04-07,longread,Секс,Жизнь
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,517117,704e61a9-b817-4403-9ffd-0f5836755e8b,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,23996,aae018e7-569b-4592-9696-09494d6c7970,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [10]:
data.drop(columns=["esb_ehr_id","patientnet_ehr_id"], inplace=True)
data.rename(columns={"пол":"gender", "возраст":"age"}, inplace=True)
data = data[data['tags'] != 'Секс']
data

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30833,9607899,CLICKED,60d599187d430d33ba5ab9c3,281078,2024-07-30,281078,1,43,1982-06-14,Существует ли синдром отмены сахара?,https://cuprum.media/science-answers/sugar-withdrawal-syndrome?from=avaapp,1347.0,2021-06-25,longread,0,Ответили по науке
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [4]:
data['ehr_id'].value_counts()

ehr_id
33123      252
1197895    237
1032916    225
1312091    216
1335858    141
          ... 
101930       1
1150456      1
1218261      1
1375897      1
100445       1
Name: count, Length: 13551, dtype: int64

In [13]:
data_interactions_count = data.groupby('ehr_id')['article_id'].count()
#users_with_5_interactions = data_interactions_count[data_interactions_count >= 5].index
users_with_5_to_29_interactions = data_interactions_count[
    (data_interactions_count >= 4) & (data_interactions_count <= 40)
].index
users_with_5_interactions = set(users_with_5_to_29_interactions)
data_with_5_interactions = data[data['ehr_id'].isin(users_with_5_interactions)]
data_with_5_interactions

data = data_with_5_interactions
df = data
df

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
12,3747821,CLICKED,62d6caac26574a515400e486,1292575,2023-07-01,1292575,2,11,2014-02-28,Как мизогиния влияет на гендерное равенство,https://cuprum.media/mentalnoe/female-misogyny?from=avaapp,1496.0,2022-07-19,longread,Ментальное,Ментальное
13,3747850,CLOSED,62d6caac26574a515400e486,1292575,2023-07-01,1292575,2,11,2014-02-28,Как мизогиния влияет на гендерное равенство,https://cuprum.media/mentalnoe/female-misogyny?from=avaapp,1496.0,2022-07-19,longread,Ментальное,Ментальное
44,3753815,CLICKED,60258f50e3c78375686c0c51,1132756,2023-07-01,1132756,1,48,1976-12-22,"​Как понять, что у вас паразиты",https://cuprum.media/science-answers/find-parasites?from=avaapp,5779.0,2021-02-11,longread,0,Ответили по науке
45,3754195,CLICKED,5f7c6c9f4d41bc762d756441,1292575,2023-07-01,1292575,2,11,2014-02-28,Волосы седеют от стресса и старости?,https://cuprum.media/proverka-sluha/why-hair-turn-gray?from=avaapp,7601.0,2020-10-06,longread,Волосы,Проверка слуха
46,3754249,CLICKED,5f7c866ed66f2a1f5b472611,1292575,2023-07-01,1292575,2,11,2014-02-28,Что не так с остеопатией,https://cuprum.media/razbor/osteopathy?from=avaapp,4524.0,2020-10-06,longread,Наука,Разбор
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30814,9602975,CLICKED,62012efa3b61f04c0a2f2725,1049448,2024-07-30,1049448,1,33,1991-11-30,Может ли у грибов развиваться устойчивость к лекарствам,https://cuprum.media/proverka-sluha/antifungal-resistance?from=avaapp,2686.0,2022-02-07,longread,Наука,Проверка слуха
30815,9603468,CLICKED,622b44d60e7b1e54fe5e5342,272451,2024-07-30,272451,1,52,1973-01-09,Cкрининг на рак груди,https://cuprum.media/spravochnik/screening-breast-cancer-sp?from=avaapp,1072.0,2022-03-11,shortread,Скрининг,Справочник
30816,9603700,CLICKED,6218dd9140e8460b6d1e326b,57714,2024-07-30,57714,1,58,1966-06-25,"Если хорошенько потрястись, это поможет снять стресс?",https://cuprum.media/proverka-sluha/shaking-off?from=avaapp,6086.0,2022-02-25,longread,Ментальное,Проверка слуха
30818,9603851,CLICKED,63e10773dbd23b1bff0b7002,263328,2024-07-30,263328,1,31,1993-09-13,Плесень на орехах и кукурузе вызывает рак печени?,https://cuprum.media/science-answers/nuts-and-cancer?from=avaapp,1455.0,2023-02-06,longread,"Еда,Онкология,Патогены",Ответили по науке


In [12]:
data_with_5_interactions['ehr_id'].value_counts()

ehr_id
1039998    39
1319100    38
1408075    38
1388141    37
256653     36
           ..
1265915     5
1391876     5
524438      5
1336071     5
1360834     5
Name: count, Length: 914, dtype: int64

In [15]:
data[data['ehr_id'] == 1391876]

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
26524,8514513,CLOSED,62f5097ab81c34e3e1003292,1391876,2024-05-13,1391876,1,27,1997-07-12,"Что делать, если кто-то случайно выпил бытовую химию",https://cuprum.media/science-answers/caustic-injury?from=avaapp,2846.0,2022-08-11,longread,"Травмы,Дом",Ответили по науке
26525,8514515,CLOSED,5fd220f00d7ff9202e7873b1,1391876,2024-05-13,1391876,1,27,1997-07-12,"Правда ли, что подагрой болеют из-за вкусной еды",https://cuprum.media/proverka-sluha/disease-of-aristocrats?from=avaapp,2847.0,2020-12-10,longread,Суставы,Проверка слуха
26526,8514517,CLOSED,5f33ec5ab45a9b7d7a63f0b0,1391876,2024-05-13,1391876,1,27,1997-07-12,"Говорят, стоя работать полезнее. Правда?",https://cuprum.media/proverka-sluha/stand-up?from=avaapp,1979.0,2020-08-15,longread,Дом,Проверка слуха
28395,8989656,CLOSED,65fc0956f6ca6f38e00baede,1391876,2024-06-16,1391876,1,27,1997-07-12,Почему после долгого чтения «видишь» текст перед глазами?,https://cuprum.media/science-answers/ostatochnoe-izobrazhenie?from=avaapp,650.0,2024-03-26,longread,"Лайфстайл,Зрение",Ответили по науке
28396,8989657,CLOSED,60d96cacb735c52c2842a6a2,1391876,2024-06-16,1391876,1,27,1997-07-12,Как питаться во время беременности,https://cuprum.media/spravochnik/food-for-pregnant?from=avaapp,861.0,2021-06-27,shortread,Еда,Справочник


In [ ]:
# Посчитаем активных пользователей
min_train_interactions = 5
min_test_interactions = 3

train_user_counts = df_train.groupby('ehr_id')['article_id'].count()
test_user_counts = df_test.groupby('ehr_id')['article_id'].count()

active_users = set(
    train_user_counts[train_user_counts >= min_train_interactions].index
) & set(
    test_user_counts[test_user_counts >= min_test_interactions].index
)

# Фильтруем train/test
df_train_filtered = df_train[df_train['ehr_id'].isin(active_users)]
df_test_filtered = df_test[df_test['ehr_id'].isin(active_users)]

In [34]:
data = data[data['action_type'] == 'CLICKED']
df = data
df

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
44,3753815,CLICKED,60258f50e3c78375686c0c51,1132756,2023-07-01,1132756,1,48,1976-12-22,"​Как понять, что у вас паразиты",https://cuprum.media/science-answers/find-parasites?from=avaapp,5779.0,2021-02-11,longread,0,Ответили по науке
51,3754559,CLICKED,62e24b5be4ff39162208fd33,1169819,2023-07-01,1169819,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
52,3754575,CLICKED,62e24b5be4ff39162208fd33,1169819,2023-07-01,1169819,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
53,3754576,CLICKED,62e24610a9c9279df00b8673,1169819,2023-07-01,1169819,1,31,1993-07-08,"Что почитать: «Мальчик, который не переставал расти» Эдвина Кёрка",https://cuprum.media/razbor/the-boy-who-never-stopped-growing?from=avaapp,347.0,2022-07-28,longread,Книги,Разбор
54,3754581,CLICKED,62e0f1c5df18e4454c047292,1169819,2023-07-01,1169819,1,31,1993-07-08,«В суете нашей жизни некогда подумать о собственном здоровье»: как предотвратить инсульт,https://cuprum.media/interview/stroke-prevention?from=avaapp,892.0,2022-07-27,interview,Диагностика,Интервью
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30776,9588070,CLICKED,5fd0dc7e5374f05648600ab2,909927,2024-07-29,909927,1,42,1982-07-30,АМГ: контроль за беременностью и яичками,https://cuprum.media/razbor/amh?from=avaapp,2129.0,2020-12-09,longread,"Репродукция,Беременность,ЭКО,ВРТ,Бесплодие",Разбор
30814,9602975,CLICKED,62012efa3b61f04c0a2f2725,1049448,2024-07-30,1049448,1,33,1991-11-30,Может ли у грибов развиваться устойчивость к лекарствам,https://cuprum.media/proverka-sluha/antifungal-resistance?from=avaapp,2686.0,2022-02-07,longread,Наука,Проверка слуха
30815,9603468,CLICKED,622b44d60e7b1e54fe5e5342,272451,2024-07-30,272451,1,52,1973-01-09,Cкрининг на рак груди,https://cuprum.media/spravochnik/screening-breast-cancer-sp?from=avaapp,1072.0,2022-03-11,shortread,Скрининг,Справочник
30816,9603700,CLICKED,6218dd9140e8460b6d1e326b,57714,2024-07-30,57714,1,58,1966-06-25,"Если хорошенько потрястись, это поможет снять стресс?",https://cuprum.media/proverka-sluha/shaking-off?from=avaapp,6086.0,2022-02-25,longread,Ментальное,Проверка слуха


In [18]:
df['weight'] = 1

# агрегируем, если нужно (например, несколько кликов одного юзера по одной статье)
df = df.groupby(['ehr_id', 'article_id'], as_index=False)['weight'].sum()

/tmp/ipykernel_23110/2462432166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = 1


In [23]:
df

,ehr_id,article_id,weight
0,3670,60f57272c17ff86423582032,1
1,3670,60f6e195a9e57f28ce4194b3,1
2,3670,61039e8a65fa090ddd554472,1
3,3670,6107a4563dfec013ad508e14,1
4,3670,624da036b341c600db685964,1
...,...,...,...
5003,1423756,61c5a180ea60a60eaf7d33ab,1
5004,1423756,61c96c55bd9dab46bf539895,1
5005,1423756,61c9728b42338d1d042eb99d,1
5006,1423756,62de5ce2ef7310a75e0bc752,1


In [19]:
from scipy.sparse import coo_matrix

# создаём маппинги
user_ids = df['ehr_id'].unique().tolist()
item_ids = df['article_id'].unique().tolist()
user_map = {u: i for i, u in enumerate(user_ids)}
item_map = {i: j for j, i in enumerate(item_ids)}

# строим COO-матрицу
rows = df['ehr_id'].map(user_map)
cols = df['article_id'].map(item_map)
data_weight = df['weight']

interactions = coo_matrix((data_weight, (rows, cols)),
                          shape=(len(user_ids), len(item_ids))).tocsr()


In [20]:
import implicit

# задаём гиперпараметры
factors = 50        # число латентных факторов
regularization = 0.01
iterations = 20
alpha = 40          # параметр масштабирования для implicit feedback

# модель
model = implicit.als.AlternatingLeastSquares(
    factors=factors,
    regularization=regularization,
    iterations=iterations,
)

# масштабируем веса (опционально)
# model.fit((interactions * alpha).astype('double'))
model.fit(interactions)


/home/mle-user/sgm_recsys/.venv_sgm_recsys/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [24]:
user_map

{3670: 0,
 16882: 1,
 17251: 2,
 18042: 3,
 21545: 4,
 22684: 5,
 23102: 6,
 24730: 7,
 25009: 8,
 25199: 9,
 27082: 10,
 30405: 11,
 31189: 12,
 31218: 13,
 32090: 14,
 34825: 15,
 40321: 16,
 44914: 17,
 46029: 18,
 46144: 19,
 46787: 20,
 57714: 21,
 61214: 22,
 66515: 23,
 66926: 24,
 67537: 25,
 68489: 26,
 73208: 27,
 76698: 28,
 78331: 29,
 93843: 30,
 99122: 31,
 102822: 32,
 105083: 33,
 107818: 34,
 110978: 35,
 121656: 36,
 128230: 37,
 130673: 38,
 131166: 39,
 131724: 40,
 132016: 41,
 134204: 42,
 139039: 43,
 140348: 44,
 141820: 45,
 142849: 46,
 143851: 47,
 147513: 48,
 148159: 49,
 149713: 50,
 151714: 51,
 153345: 52,
 155254: 53,
 157709: 54,
 158296: 55,
 158329: 56,
 159724: 57,
 163839: 58,
 166113: 59,
 170891: 60,
 173874: 61,
 174267: 62,
 175739: 63,
 175828: 64,
 177469: 65,
 182764: 66,
 188432: 67,
 199636: 68,
 208304: 69,
 221227: 70,
 230057: 71,
 232858: 72,
 234386: 73,
 240658: 74,
 244265: 75,
 247352: 76,
 248179: 77,
 256458: 78,
 256653: 79,
 25

In [26]:
# Получаем два массива: индексы артиклов и их "скор"
user_idx = user_map[44914]
item_idxs, scores = model.recommend(user_idx, interactions[user_idx], N=10)

# Конвертируем индексы обратно в реальные article_id
rec_article_ids = [item_ids[i] for i in item_idxs]

# Если вам нужны и оценки — запакуем обратно в кортежи
recommendations = list(zip(rec_article_ids, scores))

print("Топ‑10 рекомендаций (article_id, score):")
for article_id, score in recommendations:
    print(f"{article_id} (score={score:.3f})")

Топ‑10 рекомендаций (article_id, score):
60950f9c246e652272317233 (score=0.014)
5fa53a8919b05a79f9729c21 (score=0.014)
64d1dbc57c514a3e5c0e56b2 (score=0.012)
602fdf6f88c25017a2394101 (score=0.008)
614d9957cf989e64dd615294 (score=0.008)
6093a2c0367f731117790643 (score=0.008)
64d0d89a844fd8e25f06cdc6 (score=0.008)
605cb3e9c36dc26a3d5ae2f2 (score=0.008)
602e34c193a42c67331b2451 (score=0.007)
6151ddb6c9ed0c5aed549132 (score=0.007)


In [27]:
# 1. Составляем словарь из всех article_id → title
id_to_title = dict(zip(data['article_id'], data['title']))

# 2. Для своего списка рекомендаций строим список названий
recommended_titles = [id_to_title[a_id] for a_id in rec_article_ids]

recommended_titles

['Правда ли, что КОКи продлевают молодость',
 'Семь ненужных вам анализов',
 'Почему в дождь хочется спать',
 'Тест на IQ говорит, что я тупой. Верить?',
 'Диета при панкреатите',
 'Гинекологический осмотр',
 'Проверка на совместимость: стоит ли пускать питомцев в постель?',
 'Чем пахнет старость',
 'Зачем люди едят сушеные мухоморы',
 'Как лечить простуду']

In [29]:
# 1. Составляем маппинг для article_id -> title, url
id_to_title = dict(zip(data['article_id'], data['title']))
id_to_url = dict(zip(data['article_id'], data['url']))

# 2. Формируем таблицу рекомендаций
recs_df = pd.DataFrame({
    'article_id': rec_article_ids,
    'title': [id_to_title[a_id] for a_id in rec_article_ids],
    'url': [id_to_url[a_id] for a_id in rec_article_ids],
    'score': scores
})

# Сортируем по score, на всякий случай (ALS и так возвращает отсортированное)
recs_df = recs_df.sort_values('score', ascending=False).reset_index(drop=True)
recs_df.drop(columns=["score"], inplace=True)


recs_df


,article_id,title,url
0,60950f9c246e652272317233,"Правда ли, что КОКи продлевают молодость",https://cuprum.media/proverka-sluha/pills-and-eggs?from=avaapp
1,5fa53a8919b05a79f9729c21,Семь ненужных вам анализов,https://cuprum.media/razbor/unnecessary-tests?from=avaapp
2,64d1dbc57c514a3e5c0e56b2,Почему в дождь хочется спать,https://cuprum.media/razbor/pochemu-hochetsya-spat-v-dozhd?from=avaapp
3,602fdf6f88c25017a2394101,"Тест на IQ говорит, что я тупой. Верить?",https://cuprum.media/ipohondrik/mental-tests?from=avaapp
4,614d9957cf989e64dd615294,Диета при панкреатите,https://cuprum.media/spravochnik/pancreatitis-diet?from=avaapp
5,6093a2c0367f731117790643,Гинекологический осмотр,https://cuprum.media/spravochnik/ginekologicheskij-osmotr?from=avaapp
6,64d0d89a844fd8e25f06cdc6,Проверка на совместимость: стоит ли пускать питомцев в постель?,https://cuprum.media/science-answers/son-s-pitomtsem?from=avaapp
7,605cb3e9c36dc26a3d5ae2f2,Чем пахнет старость,https://cuprum.media/stesnyayus-sprosit/old-person-smell?from=avaapp
8,602e34c193a42c67331b2451,Зачем люди едят сушеные мухоморы,https://cuprum.media/stesnyayus-sprosit/dried-mushrooms?from=avaapp
9,6151ddb6c9ed0c5aed549132,Как лечить простуду,https://cuprum.media/instruction/treating-a-cold-check?from=avaapp


In [30]:
def recommend_for_user_als(user_ehr_id, N=10):
    """
    Вернуть рекомендации для одного пользователя:
    - user_ehr_id — ehr_id пользователя из данных
    - model — обученная ALS модель implicit
    - interactions — sparse матрица взаимодействий
    - user_map — словарь {ehr_id: user_idx}
    - item_ids — список всех article_id
    - data — исходный DataFrame с title и url
    - N — сколько рекомендаций вернуть

    Возвращает: DataFrame с article_id, title, url
    """

    # Проверим, что пользователь есть в маппинге
    #if user_ehr_id not in user_map:
    #    print(f"Пользователь {user_ehr_id} не найден.")
    #    return pd.DataFrame(columns=['article_id', 'title', 'url'])

    user_idx = user_map[user_ehr_id]

    # ALS рекомендации
    item_idxs, scores = model.recommend(user_idx, interactions[user_idx], N=N)

    # Индексы -> article_id
    rec_article_ids = [item_ids[i] for i in item_idxs]

    # Маппинги title + url
    id_to_title = dict(zip(data['article_id'], data['title']))
    id_to_url = dict(zip(data['article_id'], data['url']))

    # Финальный DataFrame
    recs_df = pd.DataFrame({
        'article_id': rec_article_ids,
        'title': [id_to_title.get(a_id, '') for a_id in rec_article_ids],
        'url': [id_to_url.get(a_id, '') for a_id in rec_article_ids],
        'score': scores
    }).sort_values('score', ascending=False).reset_index(drop=True)

    # Убираем score, если не нужен
    recs_df.drop(columns=['score'], inplace=True)

    return recs_df


## Загружаем ТОП

In [38]:
import pickle
import pandas as pd

# Загружаем словарь
with open('top_articles_1.pkl', 'rb') as f:
    top_articles_by_segment = pickle.load(f)

# Загружаем таблицу
articles_info = pd.read_csv('articles_info_1.csv')

# Функция та же самая
def age_group(age):
    if age < 18:
        return 0
    elif age < 30:
        return 1
    elif age < 45:
        return 2
    elif age < 60:
        return 3
    else:
        return 4

def recommend_by_demo(gender, age, N=10):
    group = age_group(age)
    ids = top_articles_by_segment.get((gender, group), [])[:N]
    recs = articles_info[articles_info['article_id'].isin(ids)]
    recs['article_id'] = pd.Categorical(recs['article_id'], categories=ids, ordered=True)
    recs = recs.sort_values('article_id')
    return recs[['article_id', 'title', 'url']].reset_index(drop=True)

# Пример
recommend_by_demo(gender=1, age=30)


/tmp/ipykernel_73868/3778301081.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs['article_id'] = pd.Categorical(recs['article_id'], categories=ids, ordered=True)


,article_id,title,url
0,62bd9b574a4176582f017bb2,Как связаны деньги и психическое здоровье?,https://cuprum.media/mentalnoe/money-mental?from=avaapp
1,5fe1ee71ca1d6006e262be11,Как правильно вытирать попу,https://cuprum.media/stesnyayus-sprosit/toilet-paper?from=avaapp
2,600992d6ad457f63cc193bb1,Что хуже для здоровья женщины: роды или аборт,https://cuprum.media/ipohondrik/childbirth-or-abortion?from=avaapp
3,63ad8d3d51bdac5ded02d4b2,Портрет донора спермы,https://cuprum.media/razbor/sperm-donors?from=avaapp
4,5fdfe5a410312331e524f0e1,Семь диетических мифов,https://cuprum.media/science-answers/ diet-myths?from=avaapp
5,628cffd79a57867ed904f5a4,Семь пустышек для женского здоровья,https://cuprum.media/razbor/womens-health?from=avaapp
6,60b74607911ec35036678cb3,Как яйцеклетка выбирает сперматозоид,https://cuprum.media/stesnyayus-sprosit/magic-fertilization?from=avaapp
7,616ecd08d2bec01cda02fd72,Сон в носках,https://cuprum.media/science-answers/sleepy-socks?from=avaapp
8,628b9967bbd64f605e639ae2,Зачем отбеливать анус,https://cuprum.media/stesnyayus-sprosit/chocolate-eye?from=avaapp
9,5f3e713a11f6f83abf1ee6e1,Что бесит врачей в общении с пациентами,https://cuprum.media/razbor/besit?from=avaapp


In [ ]:
def recommend_articles(ehr_id, gender, age):
    results = data[data["ehr_id"] == ehr_id].shape[0]
    if(results > 5):
        print("Рекомендации, основанные на истории для выбранного ehrId")
        return 
        print()
    else:
        print("Рекомендации, основанные на топе статей")
        print(recommend_by_demo(gender, age))

In [59]:
data[data["ehr_id"] == 961420]
#type(result)

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
30703,9564911,CLICKED,61c4895172cea77446440f52,961420,2024-07-27,961420,2,35,1990-05-29,"Может ли ребенок привиться, если родители против",https://cuprum.media/razbor/pupils-against-parents?from=avaapp,1484.0,2021-12-23,longread,"Вакцины,Детское здоровье",Разбор


In [70]:
recommend_articles(1312091, 1, 30)

Рекомендации, основанные на истории для выбранного ehrId
                 article_id  \
0  636e38f69cf6f367120affc2   
1  614885c39a8b7d46c3179214   
2  632082c17ef784eac403ba83   
3  628cffd79a57867ed904f5a4   
4  6411bda1c0547a0aba053b32   
5  6308a741639f4ecd1b075f23   
6  63e3b46828403e8c56022e82   
7  61b84db4d97f8c55115fc902   
8  6267d4787dfad23bbb53f353   
9  5f8992503d308179a31c21a1   

                                                                         title  \
0            Почему мужчины любят объяснять женщинам то, что им и так известно   
1                                              Тестируем трусы для менструаций   
2                                                              Что такое хилез   
3                                          Семь пустышек для женского здоровья   
4                                      Почему на ногтях появляются белые пятна   
5                                               Тревожные расстройства у детей   
6  Как неэтичные частные кл

In [8]:
from sklearn.model_selection import train_test_split

# 1. Train/Test split по взаимодействиям
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 2. Обучить ALS только на train
#  — здесь ты всё делаешь так же, просто interactions будешь строить по train_df

# 3. Для теста:
#   - сформируй словарь: {user_id: set(реальных article_id)}
test_truth = test_df.groupby('ehr_id')['article_id'].apply(set).to_dict()

# 4. Для каждого пользователя из теста: получаем top‑k
k = 10
precisions = []
recalls = []

# for user_id, true_items in test_truth.items():
#     if user_id not in user_map:
#         continue  # пользователь не попал в train

#     recs_df = recommend_for_user_als(user_id, N=k)
#     recommended_items = set(recs_df['article_id'])

#     hits = len(recommended_items & true_items)

#     precision = hits / k
#     recall = hits / len(true_items)

#     precisions.append(precision)
#     recalls.append(recall)

# print(f"Average Precision@{k}: {sum(precisions)/len(precisions):.4f}")
# print(f"Average Recall@{k}: {sum(recalls)/len(recalls):.4f}")


In [9]:
print("Train users:", train_df['ehr_id'].nunique())
print("Test users:", test_df['ehr_id'].nunique())
print("Train items:", train_df['article_id'].nunique())
print("Test items:", test_df['article_id'].nunique())


Train users: 11274
Test users: 3836
Train items: 3585
Test items: 2421


In [5]:
train_users = set(train_df['ehr_id'])
test_users = set(test_df['ehr_id'])
print("Users in both:", len(train_users & test_users))


NameError: name 'train_df' is not defined

In [26]:
data['created_at'].describe()

count                            29108
mean     2023-12-24 13:22:01.368696064
min                2023-07-01 00:00:00
25%                2023-09-19 00:00:00
50%                2023-12-16 00:00:00
75%                2024-03-27 00:00:00
max                2024-07-30 00:00:00
Name: created_at, dtype: object

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Берём твои данные
df_full = data.copy()

# Приводим к дате, если ещё не сделано
df_full['created_at'] = pd.to_datetime(df_full['created_at'])

# Выбираем дату сплита
split_date = '2024-04-01'

# Train/Test temporal split
train_df = df_full[df_full['created_at'] < split_date]
test_df  = df_full[df_full['created_at'] >= split_date]

print("Train:", train_df.shape)
print("Test:", test_df.shape)

# Готовим train взаимодействия (как раньше)
df_train = train_df[train_df['action_type'] == 'CLICKED'].copy()
df_train['weight'] = 1
df_train = df_train.groupby(['ehr_id', 'article_id'], as_index=False)['weight'].sum()

# Mapping
user_ids = df_train['ehr_id'].unique().tolist()
item_ids = df_train['article_id'].unique().tolist()

user_map = {u: i for i, u in enumerate(user_ids)}
item_map = {i: j for j, i in enumerate(item_ids)}

from scipy.sparse import coo_matrix

rows = df_train['ehr_id'].map(user_map)
cols = df_train['article_id'].map(item_map)
weights = df_train['weight']

interactions = coo_matrix((weights, (rows, cols)),
                          shape=(len(user_ids), len(item_ids))).tocsr()

# Обучаем ALS
import implicit

model = implicit.als.AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=20,
)

model.fit(interactions)

# Готовим test данные: {ehr_id: set(article_id)}
df_test = test_df[test_df['action_type'] == 'CLICKED'].copy()
test_truth = df_test.groupby('ehr_id')['article_id'].apply(set).to_dict()

# Метрики top-k
k = 10
precisions, recalls = [], []

for user_id, true_items in test_truth.items():
    if user_id not in user_map:
        continue  # пользователь не был в train

    user_idx = user_map[user_id]

    item_idxs, scores = model.recommend(user_idx, interactions[user_idx], N=k)

    rec_article_ids = [item_ids[i] for i in item_idxs]
    recommended_items = set(rec_article_ids)

    hits = len(recommended_items & true_items)

    precisions.append(hits / k)
    recalls.append(hits / len(true_items))

print(f"Average Precision@{k}: {sum(precisions)/len(precisions):.4f}")
print(f"Average Recall@{k}: {sum(recalls)/len(recalls):.4f}")


Train: (4216, 16)
Test: (1117, 16)


  0%|          | 0/20 [00:00<?, ?it/s]

Average Precision@10: 0.0013
Average Recall@10: 0.0013


In [11]:
df_test.groupby('ehr_id')['article_id'].nunique().describe()


count    3895.000000
mean        1.472144
std         1.388811
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        45.000000
Name: article_id, dtype: float64

In [21]:
hit_count = 0
total_users = 0

for user_id, true_items in test_truth.items():
    if user_id not in user_map:
        continue
    user_idx = user_map[user_id]
    item_idxs, scores = model.recommend(user_idx, interactions[user_idx], N=10)
    rec_article_ids = [item_ids[i] for i in item_idxs]
    if len(set(rec_article_ids) & true_items) > 0:
        hit_count += 1
    total_users += 1

hit_rate = hit_count / total_users
print(f"Hit Rate@10: {hit_rate:.4f}")


Hit Rate@10: 0.1429


In [20]:
df_full

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30833,9607899,CLICKED,60d599187d430d33ba5ab9c3,281078,2024-07-30,281078,1,43,1982-06-14,Существует ли синдром отмены сахара?,https://cuprum.media/science-answers/sugar-withdrawal-syndrome?from=avaapp,1347.0,2021-06-25,longread,0,Ответили по науке
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Берём твои данные
df_full = data.copy()

# Приводим к дате, если ещё не сделано
df_full['created_at'] = pd.to_datetime(df_full['created_at'])

# Выбираем дату сплита
split_date = '2024-04-01'

# Train/Test temporal split
train_df = df_full[df_full['created_at'] < split_date]
test_df  = df_full[df_full['created_at'] >= split_date]


print("Train:", train_df.shape)
print("Test:", test_df.shape)

# Посчитаем активных пользователей
min_train_interactions = 5
min_test_interactions = 3

# Готовим train взаимодействия (как раньше)
df_train = train_df[train_df['action_type'] == 'CLICKED'].copy()
df_train['weight'] = 1
df_train = df_train.groupby(['ehr_id', 'article_id'], as_index=False)['weight'].sum()

#====
train_user_counts = df_train.groupby('ehr_id')['article_id'].count()
test_user_counts = df_test.groupby('ehr_id')['article_id'].count()

active_users = set(
    train_user_counts[train_user_counts >= min_train_interactions].index
) & set(
    test_user_counts[test_user_counts >= min_test_interactions].index
)

df_train_filtered = df_train[df_train['ehr_id'].isin(active_users)]
df_train = df_train_filtered

df_test = test_df[test_df['action_type'] == 'CLICKED'].copy()

###


# Mapping
user_ids = df_train['ehr_id'].unique().tolist()
item_ids = df_train['article_id'].unique().tolist()

user_map = {u: i for i, u in enumerate(user_ids)}
item_map = {i: j for j, i in enumerate(item_ids)}

from scipy.sparse import coo_matrix

rows = df_train['ehr_id'].map(user_map)
cols = df_train['article_id'].map(item_map)
weights = df_train['weight']

interactions = coo_matrix((weights, (rows, cols)),
                          shape=(len(user_ids), len(item_ids))).tocsr()

# Обучаем ALS
import implicit

model = implicit.als.AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=20,
)

model.fit(interactions)

# Готовим test данные: {ehr_id: set(article_id)}
#df_test = test_df[test_df['action_type'] == 'CLICKED'].copy()

test_truth = df_test.groupby('ehr_id')['article_id'].apply(set).to_dict()

####




# Фильтруем train/test

df_test_filtered = df_test[df_test['ehr_id'].isin(active_users)]
df_test = df_test_filtered
###


# Метрики top-k
k = 10
precisions, recalls = [], []

# Только те, которые есть в train
valid_items = set(df_train['article_id'].unique())

df_test = df_test[df_test['article_id'].isin(valid_items)]
test_truth = df_test.groupby('ehr_id')['article_id'].apply(set).to_dict()


for user_id, true_items in test_truth.items():
    if user_id not in user_map:
        continue  # пользователь не был в train

    user_idx = user_map[user_id]

    item_idxs, scores = model.recommend(user_idx, interactions[user_idx], N=k)

    rec_article_ids = [item_ids[i] for i in item_idxs]
    recommended_items = set(rec_article_ids)

    hits = len(recommended_items & true_items)

    precisions.append(hits / k)
    recalls.append(hits / len(true_items))

print(f"Average Precision@{k}: {sum(precisions)/len(precisions):.4f}")
print(f"Average Recall@{k}: {sum(recalls)/len(recalls):.4f}")

Train: (22124, 16)
Test: (6984, 16)


  0%|          | 0/20 [00:00<?, ?it/s]

Average Precision@10: 0.0143
Average Recall@10: 0.0357


In [7]:
df

NameError: name 'df' is not defined

In [14]:
import random

positive_pairs = df[['ehr_id', 'article_id']].drop_duplicates()
positive_pairs['target'] = 1

# Негативные примеры
all_articles = set(df['article_id'].unique())
negative_samples = []

for user in df['ehr_id'].unique():
    user_clicked = set(df[df['ehr_id'] == user]['article_id'])
    negative_pool = list(all_articles - user_clicked)
    sampled_negatives = random.sample(negative_pool, min(10, len(negative_pool)))
    
    for article in sampled_negatives:
        negative_samples.append({'ehr_id': user, 'article_id': article, 'target': 0})

negative_df = pd.DataFrame(negative_samples)
negative_df

,ehr_id,article_id,target
0,1292575,613b0f0de0151e512a19bde7,0
1,1292575,63ea42aaa6594aa87e07c4f2,0
2,1292575,5fe32445b988703cdb3b9961,0
3,1292575,626a9fef6d01df32ed3b6ca3,0
4,1292575,61683e52dd40ee5ce04bed68,0
...,...,...,...
13865,1231126,642a98a65da03e7453072492,0
13866,1231126,5fbbdd83cd27696fba15b561,0
13867,1231126,6380bef91f48ed43450eb212,0
13868,1231126,63ee3e646e7e63180d013d42,0


In [18]:
import pandas as pd
import numpy as np
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random


In [15]:
# Позитивные
df_clicks = data[data['action_type'] == 'CLICKED']
df_clicks = df_clicks[['ehr_id', 'article_id', 'created_at']].drop_duplicates()
df_clicks['target'] = 1

# Все статьи
all_articles = data['article_id'].unique().tolist()

# Негативные
negatives = []

for user, group in df_clicks.groupby('ehr_id'):
    pos_items = set(group['article_id'])
    neg_pool = list(set(all_articles) - pos_items)
    sampled_neg = random.sample(neg_pool, min(10 * len(pos_items), len(neg_pool)))

    for article in sampled_neg:
        negatives.append({'ehr_id': user, 'article_id': article, 'target': 0})

df_neg = pd.DataFrame(negatives)

# Объединяем
df_all = pd.concat([df_clicks[['ehr_id', 'article_id', 'target']], df_neg], ignore_index=True)


In [16]:
# Кол-во кликов по статье (популярность)
article_popularity = df_clicks.groupby('article_id').size().to_dict()
df_all['article_popularity'] = df_all['article_id'].map(article_popularity).fillna(0)

# Кол-во кликов у пользователя (активность)
user_activity = df_clicks.groupby('ehr_id').size().to_dict()
df_all['user_click_count'] = df_all['ehr_id'].map(user_activity).fillna(0)

# Фичи
feature_cols = ['article_popularity', 'user_click_count']


In [20]:
train_df, test_df = train_test_split(df_all, test_size=0.2, random_state=42, stratify=df_all['ehr_id'])

train_df = train_df.sort_values('ehr_id').reset_index(drop=True)
test_df = test_df.sort_values('ehr_id').reset_index(drop=True)

# Группы для ранжирования (по пользователям)
train_groups = train_df.groupby('ehr_id').size().values
test_groups = test_df.groupby('ehr_id').size().values

train_pool = Pool(data=train_df[feature_cols], 
                  label=train_df['target'], 
                  group_id=train_df['ehr_id'])

test_pool = Pool(data=test_df[feature_cols], 
                 label=test_df['target'], 
                 group_id=test_df['ehr_id'])


In [21]:
model = CatBoostRanker(
    iterations=300,
    learning_rate=0.1,
    depth=6,
    loss_function='YetiRank',
    verbose=50,
    random_seed=42
)

model.fit(train_pool)


0:	total: 151ms	remaining: 45.2s
50:	total: 2.69s	remaining: 13.2s
100:	total: 4.96s	remaining: 9.78s
150:	total: 7.18s	remaining: 7.09s
200:	total: 9.37s	remaining: 4.61s
250:	total: 11.6s	remaining: 2.26s
299:	total: 13.8s	remaining: 0us


In [22]:
def map_at_k(df, model, k=10):
    scores = model.predict(df[feature_cols])
    df = df.copy()
    df['score'] = scores

    map_total, user_count = 0, 0
    for user, group in df.groupby('ehr_id'):
        group_sorted = group.sort_values('score', ascending=False).head(k)
        rel = group_sorted['target'].values

        if rel.sum() == 0:
            continue  # пропускаем, если нет релевантных

        precisions = [rel[:i+1].sum() / (i+1) for i in range(len(rel)) if rel[i] == 1]
        if precisions:
            map_total += sum(precisions) / min(rel.sum(), k)
            user_count += 1

    return map_total / user_count if user_count else 0


In [23]:
score = map_at_k(test_df, model, k=10)
print(f"MAP@10: {score:.4f}")


MAP@10: 0.4889


In [25]:
import pandas as pd
import numpy as np

def age_group(age):
    if age < 18:
        return 0
    elif age < 30:
        return 1
    elif age < 45:
        return 2
    elif age < 60:
        return 3
    else:
        return 4

def apk(actual, predicted, k=10):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k)

def hit_rate_at_k(actual, predicted, k=10):
    return int(bool(set(predicted[:k]) & actual))

def recall_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    return len(set(predicted[:k]) & actual) / len(actual)

def evaluate_segment_model(data, top_articles_by_segment, k=10):
    # Только клики
    clicked = data[data['action_type'] == 'CLICKED'].copy()

    # Выбираем пользователей с 5–40 кликами для теста
    user_click_counts = clicked['ehr_id'].value_counts()
    valid_users = user_click_counts[(user_click_counts >= 5) & (user_click_counts <= 40)].index

    test_df = clicked[clicked['ehr_id'].isin(valid_users)].copy()

    # Добавляем возрастную группу
    test_df['age_group'] = test_df['age'].apply(age_group)
    test_df['target'] = 1  # метка — клик

    # Группируем реальные клики по пользователю
    ground_truth = (
        test_df.groupby('ehr_id')['article_id']
        .apply(set)
        .to_dict()
    )

    # Получаем рекомендации по демо-сегменту
    user_recs = {}
    for _, row in test_df[['ehr_id', 'gender', 'age']].drop_duplicates().iterrows():
        ehr_id = row['ehr_id']
        gender = row['gender']
        age = row['age']
        group = age_group(age)
        recs = top_articles_by_segment.get((gender, group), [])
        user_recs[ehr_id] = recs[:k]

    # Метрики
    apk_scores = []
    hit_scores = []
    recall_scores = []

    for ehr_id, actual in ground_truth.items():
        predicted = user_recs.get(ehr_id, [])
        apk_scores.append(apk(actual, predicted, k=k))
        hit_scores.append(hit_rate_at_k(actual, predicted, k=k))
        recall_scores.append(recall_at_k(actual, predicted, k=k))

    print(f"Оценка демо-модели на {len(ground_truth)} пользователях:")
    print(f"MAP@{k}: {np.mean(apk_scores):.4f}")
    print(f"HitRate@{k}: {np.mean(hit_scores):.4f}")
    print(f"Recall@{k}: {np.mean(recall_scores):.4f}")


In [26]:
# Перед этим должен быть загружен полный data (train + test)
evaluate_segment_model(data, top_articles_by_segment, k=10)


NameError: name 'top_articles_by_segment' is not defined